# Assignment 09

## Association Models  

## CSCI E-96

### Steve Elston

## Introduction     

Association models are widely used to find **association rules** between items in **baskets**. A basket is a set of items assocaited with a single event such as a purchase by a customer at the store. The goal is to find rules for **frequent item sets** that associate common items found with high probability in the baskets.  

There are a number of widely used algorithms for finding frequent itemsets. In this notebook you will apply two algorithms, **apriori** and **fp-growth**. To execute the code in this notebook you will need to install the mlxtend package, following [these instructions](https://rasbt.github.io/mlxtend/installation/).    

To get started, execute the code in the cell below to import the packages required for this notebook.   

In [ ]:
import pandas as pd  
import numpy as np
from os import path
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

%matplotlib inline

## Loading the Dataset

The dataset used here is a derivative of the more complex data used for the [Kaggle Instacrt Market Basket Analysis competition](https://www.kaggle.com/competitions/instacart-market-basket-analysis/data?select=orders.csv.zip) held in 2017. The orginal dataset was much larger and contained files and features we will not use here.     

There are two files from the Kaggle competition files required for this notebook:     
- orders_products_priors.csv contins the orders (baskets) of Intacart orders. This file is over 500 MB.    
- products.csv constins information to map item ids to english lanaguge product descriptions.            

You will need to download these two files from the Kaggle contest site by the following steps:   
1. In you file system, find or create a directory where you will store these files.     
2. Go to the `Data` tab on the Kaggle contest site and download the required files. You will need to sign in to Kaggle to do so.   
3. Set the **relative `PATH` variable** in the code cell below for the path from where you run the notebook to where you have stored the data files.       

In [ ]:
PATH='../data/instacart-market-basket-analysis'

### Load the product id files

With the files download you are now ready to import the files into the notebook. The code in the cell below does the following:   
1. Loads the .csv file.    
2. Set the index of the data frame to the product id.
3. Display the head of the data frame.  

In [ ]:
product_names = pd.read_csv(path.join(PATH, 'products.csv'))
product_names.set_index('product_id', inplace=True)
product_names.head(10)

Only the index, product id, and product name column will be used here.   

### Load the basket dataset

Next execute the code in the cell below to import the large basket dataset.   

In [ ]:
baskets_data = pd.read_csv(path.join(PATH, 'order_products__prior.csv'))
print(baskets_data.head(20))

Examine the data displayed. There are multiple rows for each order id (basket). The product ids are in the second column. The other columns are not used here.   

### Reformat the basket dataset    

Reformatting of the data is required before we can apply the algorithms in the [mlxtend.frequent_patterns](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/) subpackage.   

The data includes over 3 million baskets with 50 thousand possible items. If repersented as a dense array there would be over 150 million values, mostly 0s. Such a representation will consume significant about of memory. Therefore the dataset will be represented by a sparse matrix. The code in the cell below performs the following transformations of the basket data:    
> 1. Transform the data frame into a list of lists. The top level is a list of the baskets, determined by unique order id. The second level lists are the product ids for each unique order.     
> 2. The [mlxtend.preprocessing.transactionencoder](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.preprocessing/#transactionencoder) is apply to the input data frame to create a sparse representation of the products in the baskets. This data structure is then transformed into a [sparse Pandas data frame](https://pandas.pydata.org/docs/user_guide/sparse.html).     
> 3. The column names are set to the product ids.       

Execute the code.  

In [ ]:
## Create a list of lists, containing the baskets  
basket_list = baskets_data.loc[:,['order_id', 'product_id']].groupby('order_id')['product_id'].apply(list).tolist()

## 
te = TransactionEncoder()
oht_ary = te.fit(basket_list).transform(basket_list, sparse=True)
sparse_basket_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)

## Add product identifiers as column names for the data frame.   
sparse_basket_df.columns = [str(i) for i in sparse_basket_df.columns]
sparse_basket_df.head(20)

The `head()` method displays the 20 rows of the dataframe in a dense format. Notice that all the values displayed are 0s. There are very few products in the baskets.   

## Exploring the Dataset     

Before applying any association rule minning algorithms we will first explore the dataset.    

As a first step, execute the code in the cell below to display a summary of the data characteristics.   

In [ ]:
print('Unique number of orders = ' + str(len(baskets_data.order_id.unique())))
print('Unique number of products = ' + str(len(baskets_data.product_id.unique())))
print('Average number of products per order = ' + str(baskets_data.shape[0]/len(baskets_data.order_id.unique())))

> **Exercise 9-1:** You can see there are more baskets than unique items. Also, the baskets are generally quite small on average. This small number of items per basket is why a spare data frame is an efficient representation of these data. Ignoring overhead, such as hash structures, approximaely how much memory compression does the sparse array achieve as compared to the alternative dense array? Compute the answer in the cell below.      

In [ ]:
print('memory is compressed by a factor = ' + str(round(49677/10.1, 6)))
print('Or, memory compression is = ' + str(round(10.1/49677, 6)) + ' to 1')

### Support analysis    

You will now create some plots to explore the distribution of the suppot of the items in the baskets. Recall that for a database $D$ contining items in the set $\mathbf{I} = {I_1, I_2,..., I+N }$, the support for item $I_i, Sup_{T}(I_i),$ by rule $T$ is: 

$$Sup_{T}(I_i) = \frac{|\{ I_i \subseteq D\}|}{|D|}$$

The code in the cell below computes the D support of all the products used in the baskets in a descending order of frequency. Some summary statistics are then computed and displayed. Execute this code and examine the results.   

In [ ]:
num_baskets = sparse_basket_df.shape[0]
item_frequency = [sparse_basket_df.loc[:,i].sum(axis=0)/num_baskets for i in sparse_basket_df.columns]
item_frequency.sort(reverse=True)

median_support = np.median(item_frequency)
mean_support = np.mean(item_frequency)
print('Mean support = ' + str(round(mean_support, 6)))
print('Median support = ' + str(round(median_support, 7)))

Notice that the mean support is a reasonable number, whereas the median support is very small. This indicates that many of the products are rarely ordered.       

The code in the cell below displays the fequency distribution of item support for the most frequent 200 items in the baskets. A proposed minimum support level is shown by a horizontal red line. Execute this code and examine the result.   

In [ ]:
def plot_item_fequency(item_frequency, end_sample, ax, min_support, start_sample=1): 
    ax.plot(list(range(start_sample, end_sample)), item_frequency[start_sample:end_sample])
    ax.hlines(min_support, start_sample, end_sample, color='red')
    ax.set_xlabel('Item')
    ax.set_ylabel('Log support')
    ax.set_title('Log support by item')
    ax.set_yscale('log')


min_support=0.012
end_sample = 200
fig, ax = plt.subplots(figsize=(6,5))
plot_item_fequency(item_frequency, end_sample, ax, min_support)

There appears to be a bit of a break in the above curve around 18. To investigate this point further, execute the code in the cell below to display side by side plots of subsets of the log support values.     

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8,3))

end_sample=18
plot_item_fequency(item_frequency, end_sample, ax[0], min_support)
start_sample=19 
end_sample=200
plot_item_fequency(item_frequency, end_sample, ax[1], min_support, start_sample=start_sample)

> **Exercise 9-2:** The support, and therefore frequency, of the items decline in a super-logarithmic manner. That is, faster than logarithmic. Even for the first 18 items as well as for items 19-200. Describe what this observaqtions means in terms of contents of the baskets.         

> **Answer:**     

### Frequncy of basket size

Another view on the basket data is the frequency of the number of items in the baskets. A visualizaiton will give an idea of the distribution of the number of items in customers' baskets. 

There is a difficulty with computing the frequency of basket size for the sparse data frame. The number of items in a basket is the sum of along the row of the data frame. However, expanding the entire sparse data frame to do this calculation for all baskets requires and excessive amount of memory. 

A solution to the excessive memory problem is to operate a row at a time. This approach trades lower memoty consumption with less efficient computing. In principle, this operation can be performed with MapRuduce. The approach we adopt here is to use a **representative random sample** of the baskets. 

The code in the cell below randomly (Bernoulli) samples a fraction of $0.005$ or about $16,000$ rows of the sparse data frame. The basket size is computed for each of these sampled rows. Execute the code in the cell below to compute the sample.       

In [ ]:
sample_size=int(0.005*sparse_basket_df.shape[0])
index_sample = np.random.choice(list(range(sparse_basket_df.shape[0])), size=sample_size)
print('Size of the sample = ' + str(len(index_sample)))
basket_size_samples = sparse_basket_df.iloc[index_sample,:].sum(axis=1)

To display some summary statistics and a histogram of the sample of basket sizes execute the code in the cell below.   

In [ ]:
mean_basket = np.mean(basket_size_samples)
median_basket = np.median(basket_size_samples)
print('Mean basket size = ' + str(round(mean_basket, 1)))
print('Median basket size = ' + str(round(median_basket, 1)))
print('Minimum size = ' + str(np.min(basket_size_samples))  + '  Maximum size = ' + str(np.max(basket_size_samples)))

plt.hist(basket_size_samples, bins=100, color='lightgray');
plt.vlines(mean_basket, 0, 1200, color='red', label='mean');
plt.vlines(median_basket, 0, 1200, color='green', label='median');
plt.legend();
plt.xlabel('Number of items in basket');
plt.ylabel('Frequency');
plt.title('Basket size frequency');

Notice the distribution of basket sizes is strongly right skewed. The mode of the basket size distribution is less than the median, which in turn is less than the mean. In other words, most baskets are quite small, as small as a single unique item, and only a few are large, at least 96 unique items. Consequently, one should expect only few association rules to arrise for any reasonable support threshold.   

## Apply the Apriori Algorithm  

You will now apply the [apriori algorithm from the mlxtend package](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/#apriori) to the basket data. The minimum support used in the same as shown in the foregoing plots. To do so, execute the code in the cell below. 

In [ ]:
item_sets = apriori(sparse_basket_df, min_support=min_support, use_colnames=True, verbose=1)

To display the items and itemsets, the english lanugage names need to be joined with the support. The code in the cell below expands the item set tuples using the product ids to create a data frame with support and item set names as the columns. Execute the code to display the result.    

In [ ]:
def display_item_sets(set, product_names): 
    prod_list = product_names.product_name
    named_sets =[]
    for item in set.itemsets: 
        named_sets.append([prod_list[int(id)] for id in list(item)])
    return pd.DataFrame({'support':set.support,
                         'itemsets':named_sets}).sort_values('support', ascending=False).reset_index()

pd.set_option('display.max_rows', None)
named_item_sets = display_item_sets(item_sets, product_names)        
named_item_sets

The frequent item sets need to be transformed into the corresponding association rules. The rules can create the item sets by permuting which items are the antecedents and consequents. Execute the code in the cell below to display the derived rules.    

In [ ]:
association_rules(named_item_sets, min_threshold=min_support)

> **Exercise 9-3:** Examine the list of items and item sets displayed above. Discribe the downward closer property of sets relates the idividual items, or antecedents, and the item sets, or consequents?       

> **Answer:**     

## The FP-Growth Algorithm  

You will now apply the [fp-growth algorithm from the mlxtend package](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/#fpgrowth) to the basket data. You will use the same support threshold as before. Execute the code in the cell below to apply the fp-growth algorithm.      

In [ ]:
fp_item_sets = fpgrowth(sparse_basket_df, min_support=min_support, use_colnames=True, verbose=1)

Examine the verbose outout from the model code. The fp-growth algorithm iterates though all the items with sufficient support. Notice that in 3 cases itemsets with two item consequents are found from an antecedent. These antecedents are then followed by the rules found.         

To see the item sets with sufficient support found by the fp-growth algorthm execute the code in the cell below.

In [ ]:
fp_named_item_sets = display_item_sets(fp_item_sets, product_names)        
fp_named_item_sets

To display the association rule set found by the fp-growth algorthm execute the code in the cell below.    

In [ ]:
association_results = association_rules(named_item_sets, min_threshold=min_support)
association_results

> **Exercise 9-4:** Compare the item set list and the association rules found with the fp-growth algorithm with those found using the apriori algorithm. How can you explain the similarity in terms of the support criteria used by both algorithms?         

> **Answer:**     

## Evaluation of Results  

THe final question to address how can one understand and apply these association rules. To do so, it is necessary to consider multiple evaluation metrics. As you have seen, association rules are minned using support as the metric. However, when faced with large sets of association rules it is necssary to filter the set to the most effective or important rules. Any of the evaluation metrics can be used as filtering criteria, including:    
- Lift     
- Confidence     
- Conviction     
- Leverage    

Multiple filtering criteria are often applied to large association rule sets.     

> **Exercise 9-5:** To understand these results, start by sorting and displaying the `association_results` table produced with the fp-growth algorithm. To do so, perform a nested sort on the `lift` and `conviction` columns in descending order using the [pandas.DataFrame.sort_values](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) method. Apply [pandas.DataFrame.reset_index](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html) to the sorted result.  

In [ ]:
## Put your code below



> Examine the resulting table and answer these quentstions:   
> 1. Notice that the first two rows of the sorted table have essentially the same support, lift and leverage, and yet have very different values of conviction. How can you explain these results in terms of the values of antecedent support and consequent support?        
> 2. For the growcer, will it be more effective to offer an incentive to buy a bag of organic bannanas to purchasers of organic raspberries or vice versa and why?  

> **Answers:**    
> 1.       
> 2.       

> **Exercise 9-6:** You will now filter the sorted association rule set. In this case you will use several filter criteria to find the most effective association rules. You can perform this task with one line of code by appling the following filter criteria to the association rule dataframe. The filter criteria are intended to find rules that are significnatly better than random guessing. Now, apply these three filters:    
> 1. $lift >= 2.0$    
> 2. $conviction >= 1.1$   
> 3. $leverage <= 0.01$

In [ ]:
## Put your code below





> Examine these results and answer the following questions:  
> 1. Notice that the last rule has the highest support and confidence, but yet has the lowest lift. How can you explain this situation in terms of the other metrics?        
> 2. The first three rules all have identical support and nearly the same leverage. Which metrics differentiate these rules and why?    

> **Answers:**   
> 1.      
> 2.     

#### Copyright 2023, Stephen F Elston. All rights reserved.  